In [1]:
#%pip install langchain gradio chromadb pdfminer.six langchain-community notebook langchain_ollama langchain_chroma pymupdf openai langsmith langchain[openai] langchain-community openevals

In [2]:
import os
from dotenv import load_dotenv
import gradio as gr
from langchain_community.document_loaders import DirectoryLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# 2. Evaluation
from langsmith import Client, wrappers
from openevals.llm import create_llm_as_judge
from openevals.prompts import CORRECTNESS_PROMPT
from openai import OpenAI

/home/adam/Doc/pdfMind/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
load_dotenv()
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
# 1. Configuration
DATA_PATH = "data"
LLM_MODEL_NAME = "mistral-nemo:latest"
EMBED_MODEL_NAME = "nomic-embed-text"
CHROMA_PATH = f"/home/adam/Documents/adam/pdfMind/chroma_db/{EMBED_MODEL_NAME}"

In [5]:
print("Creating embeddings and vector store (this may take a moment)...")
embeddings = OllamaEmbeddings(model=EMBED_MODEL_NAME)
llm = ChatOllama(model=LLM_MODEL_NAME)

Creating embeddings and vector store (this may take a moment)...


In [6]:
# 2. Load Documents
loader = DirectoryLoader(DATA_PATH, glob="*.pdf", loader_cls=PyMuPDFLoader)
documents = loader.load()

In [7]:
# 2. Fonction de nettoyage optimisée pour le Français
import re
def clean_french_text(text):
    text = text.replace('\xa0', ' ').replace('\n', ' ')
    text = text.replace("’", "'").replace("‘", "'")
    text = re.sub(r'(\w)-\s+(\w)', r'\1\2', text)
    text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text
cleaned_documents = []
for doc in documents:
    doc.page_content = clean_french_text(doc.page_content)
    if len(doc.page_content) > 20:
        cleaned_documents.append(doc)

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "\n", ". ", " ", ""])
chunks = text_splitter.split_documents(cleaned_documents)
print(f"Split into {len(chunks)} chunks.")

Split into 305 chunks.


In [9]:
# Create Chroma vector store from documents
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=CHROMA_PATH
)

In [10]:
retriever = vector_store.as_retriever(type="similarity", search_kwargs={"k": 5})

In [40]:
from langsmith import Client, traceable

template = """
RÔLE :
Tu es un consultant expert en qualifications du bâtiment (Qualibat, RGE, Normes).
Ta mission est d'expliquer les documents techniques fournis de manière pédagogique, structurée et synthétique.

RÈGLES DE RÉDACTION (À SUIVRE IMPÉRATIVEMENT) :

1. STRUCTURE VISUELLE :
   - Commence toujours par une phrase d'introduction qui pose le contexte global.
   - Utilise des **titres de sections** clairs pour séparer les thématiques.
   - Utilise systématiquement des listes à puces (•) pour énumérer les détails.

2. PÉDAGOGIE ET DÉTAILS :
   - Si la question porte sur une **nomenclature ou un code** : Décortique la logique (ex: 1er chiffre = Famille). Donne un exemple concret (comme le code 2111 ou autre présent dans le contexte) pour illustrer.
   - Si la question porte sur des **règles/normes** : Cite précisément les références (ex: NF X 46-010) et les durées de validité.
   - Mets en **gras** les termes techniques importants, les chiffres clés et les concepts définis.

3. TON :
   - Professionnel, instructif et précis.
   - Ne dis jamais "D'après le contexte", intègre l'information comme une connaissance établie.

4. CONTRAINTE : Si la réponse n'est pas dans le contexte, dis "Je ne sais pas". N'invente rien.

CONTEXTE DOCUMENTAIRE :
{context}


RÉPONSE STRUCTURÉE :
"""


@traceable()# langchain Retriever will be automatically traced
def rag_bot(question: str) -> dict:
    docs = retriever.invoke(question)
    docs_string = "".join(doc.page_content for doc in docs)
    instructions = f"""You are a helpful assistant who is good at analyzing source information and answering questions.
       Use the following source documents to answer the user's questions.
       If you don't know the answer, just say that you don't know.
       Use three sentences maximum and keep the answer concise.

Documents:
{docs_string}"""
    # langchain ChatModel will be automatically traced
    ai_msg = llm.invoke([
            {"role": "system", "content": instructions},
            {"role": "user", "content": question},
        ],
    )
    return {"answer": ai_msg.content, "documents": docs}


In [12]:
def format_docs(docs):
        return "\n\n".join(doc.page_content for doc in docs)

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)


In [13]:
response = rag_chain.invoke("Quelles sont les 9 Famille d'activités ?")
answer = response["answer"]
context_docs = response["context"]
print(answer)
print("*"*100)

for doc in context_docs:
    print(doc.page_content)
    print("*"*100)

Les documents techniques fournis font état de différentes qualifications, chacune correspondant à une famille d'activité spécifique dans le domaine du bâtiment. Voici la liste des neuf familles d'activités telles qu'elles peuvent être déduites des informations présentées :

1. **Reprises en sous-œuvre** : Cette famille d'activité concerne les travaux de reprises en sous-œuvre, que ce soit pour des profondeurs supérieures ou inférieures à 3 mètres. La qualification **1223** est associée à cette famille.
2. **Micro-pieux et pieux dans le sol** : Cette famille comprend l'étude et la réalisation de micropieux (**1231**) ainsi que celle de tous types de pieux (**1233**), quelle que soit la profondeur et la méthode d'exécution.
3. **Parois moulées dans le sol** : Cette famille concerne l'étude et la réalisation de travaux de parois moulées dans le sol avec emploi de boues appropriées (**1243**).
*************************************************************************************************

In [14]:
import pandas as pd

In [37]:
eval_data = pd.read_csv('data/tests/samples.csv', sep=";")


In [38]:
dataset_name = "Mon Dataset d'Evaluation"

# Vérifie si le dataset existe déjà pour éviter les doublons
if not client.has_dataset(dataset_name=dataset_name):
    client.create_dataset(dataset_name=dataset_name)

# Transformation sécurisée (marche mieux si tu as un DataFrame ou une liste)
client.create_examples(
    inputs=[{"question": q} for q in eval_data["question"]],
    outputs=[{"answer": a} for a in eval_data["answer"]],
    dataset_name=dataset_name
)

{'example_ids': ['f074848b-0a25-4b3e-be54-53ffe8fcfdb9',
  '1fedb6ad-c31e-4458-8eaf-9e5ce6c8ab35',
  '75ce6f6f-0486-4503-8500-1ee1fa415c74',
  'b79b3807-8c2b-44ae-b020-ea2e126045eb',
  '5ade512c-c592-407f-9bd8-8d7246f38526',
  '6317de83-e89c-4f6e-8fa1-b8b883a8ba8a',
  '71033f48-b2e2-4f17-af81-e0a7b381835e',
  'c5bfd178-f807-4172-9745-fde52211c01e',
  '252e5982-7375-4ec8-905b-f2fffcd3ead9',
  '3c6a15c6-4cc8-49a5-b908-728f1833ac24',
  '6aa3d720-4b5a-4b60-8979-cde348f8f6d7',
  'f2b2cbb7-9610-47cd-890f-b62f5337b59a'],
 'count': 12}

In [ ]:
from langsmith import Client
client = Client()